In [1]:
pip install ddddocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 75.9 MB 1.5 MB/s 
     |████████████████████████████████| 4.5 MB 28.1 MB/s 
     |████████████████████████████████| 46 kB 2.1 MB/s 
     |████████████████████████████████| 86 kB 3.2 MB/s 


In [ ]:
import os
import re
import requests
import sys
import json
from bs4 import BeautifulSoup
import ddddocr
import pandas as pd
import numpy as np
import six
import csv

In [ ]:
# 重複執行直到驗證通過
ocr = ddddocr.DdddOcr()
while True:
    # 宣告Session物件
    session = requests.Session()
    # 發出http請求
    resp = session.get('https://bsr.twse.com.tw/bshtm/bsMenu.aspx', verify=False)

    # 如果回應200代表請求成功
    if resp.status_code == 200:
        # 使用 BeautifulSoup 解析 html
        soup = BeautifulSoup(resp.text, 'lxml')
        # 取得所有 'input' 物件
        nodes = soup.select('input')
        # 將欲傳送的資料存至 dictionary 當中
        params = {}
        for node in nodes:
            name = node.attrs['name']

            if name in ('RadioButton_Excd', 'Button_Reset'):
                continue

            if 'value' in node.attrs:
                params[node.attrs['name']] = node.attrs['value']
            else:
                params[node.attrs['name']] = ''

        # 取得圖片物件
        captcha_image = soup.select('#Panel_bshtm img')[0]['src']
        m = re.search(r'guid=(.+)', captcha_image)
        if m is None:
            exit(1)

        # 圖片存檔
        imgpath = '%s.jpg' % m.group(1)
        url = 'https://bsr.twse.com.tw/bshtm/' + captcha_image
        resp = requests.get(url, verify=False)
        if resp.status_code == 200:
            with open(imgpath, 'wb') as f:
                f.write(resp.content)
            os.system('open ' + imgpath)
            f.close()

        with open(imgpath, 'rb') as f:
            img_bytes = f.read()
        
        #利用OCR辨識圖片
        vcode = ocr.classification(img_bytes)
        f.close()

        # 圖片刪除
        try:
            os.remove(imgpath)
        except:
            pass
        params['CaptchaControl1'] = vcode

        #股票代碼
        params['TextBox_Stkno'] = '2317'
        print(json.dumps(params, indent=2))

        #送出資料
        resp = session.post('https://bsr.twse.com.tw/bshtm/bsMenu.aspx', data=params)
        if resp.status_code != 200:
            print('任務失敗: %d' % resp.status_code)
            continue

        #利用BeautifulSoup解析HTML
        soup = BeautifulSoup(resp.text, 'lxml')
        nodes = soup.select('#HyperLink_DownloadCSV')
        if len(nodes) == 0:
            print('任務失敗，沒有下載連結')
            continue

        #下載卷商分點進出CSV
        resp = session.get('https://bsr.twse.com.tw/bshtm/bsContent.aspx')
        with open('output.csv', 'w', encoding="utf-8") as f:
            writer = csv.writer(f)
            for resp_line in resp.text.split('\n')[2:]:
                a = resp_line.split(',')
                print(a)
                if len(a) >= 4:
                    a[1] = re.sub(r'[0-9A-Za-z ]+', '', a[1])
                    if len(a) >= 10:
                        a[7] = re.sub(r'[0-9A-Za-z ]+', '', a[7])
                    writer.writerow(a)
            f.close()

        if resp.status_code != 200:
            print('任務失敗，無法下載卷商分點進出 CSV')
            continue
        break


欢迎使用ddddocr，本项目专注带动行业内卷，个人博客:wenanzhe.com
训练数据支持来源于:http://146.56.204.113:19199/preview
爬虫框架feapder可快速一键接入，快速开启爬虫之旅：https://github.com/Boris-code/feapder
谷歌reCaptcha验证码 / hCaptcha验证码 / funCaptcha验证码商业级识别接口：https://yescaptcha.com/i/NSwk7i
{
  "__VIEWSTATE": "/wEPDwUJOTAxMjkxMjg3D2QWAgIDD2QWBAIBD2QWAmYPZBYEZg9kFgJmDxAPFgYeB0NoZWNrZWRnHglGb3JlQ29sb3IKAB4EXyFTQgIEZGRkZAIBD2QWAgIBDw8WBB8BCgAfAgIEZGQCAw9kFgQCAw8PFgIeBFRleHRlZGQCCQ8PFgIeB1Zpc2libGVoZGQYAgUeX19Db250cm9sc1JlcXVpcmVQb3N0QmFja0tleV9fFgMFElJhZGlvQnV0dG9uX05vcm1hbAUQUmFkaW9CdXR0b25fRXhjZAUQUmFkaW9CdXR0b25fRXhjZAUPQ2FwdGNoYUNvbnRyb2wxDwUkNzk3YTNjMmUtMTdhNS00NTUzLTk1NzMtZjIzMTJkYzkyMmY2ZFMgmFvkPUK2/50ozs8ShgI=",
  "__VIEWSTATEGENERATOR": "AA1F01CB",
  "__EVENTVALIDATION": "/wEdAAZwKc17eYx/rF361Rbd857afMmuxAJNAJcNkRRsVeJwwqKURZav/+YrVMqaWE2hvMxJwK4Ohf3nRgXHkFhQen1PRSciNdPYWmINCrip1wqw01PJCA0uL9aE2sjICZqv6GrhXZ/5VG0f7KGJnjdorshr",
  "RadioButton_Normal": "RadioButton_Normal",
  "TextBox_Stkno": "2317",
  "CaptchaControl1": "ryd6f",
  "

In [ ]:
data = pd.read_csv("output.csv")
combine_df = pd.DataFrame()
print(data.head())

   序號    券商     價格  買進股數    賣出股數  Unnamed: 5  序號.1  券商.1   價格.1  買進股數.1  \
0   1  合　　庫  102.5  5000       0         NaN     2  合　　庫  103.0    1230   
1   3  合　　庫  103.5     0    4000         NaN     4  合庫台中  102.0    1000   
2   5  合庫台中  102.5  2000    2000         NaN     6  合庫台中  103.0   11286   
3   7  合庫台中  103.5  2000  494000         NaN     8  合庫台南  103.0     110   
4   9  合庫高雄  102.5    20       0         NaN    10  合庫高雄  103.0       0   

   賣出股數\r  
0    2000  
1       0  
2   17697  
3    2000  
4    2100  


In [ ]:
df_left = data[['券商','價格','買進股數','賣出股數']]
df_right = data[['券商.1','價格.1','買進股數.1','賣出股數\r']]
df_right = df_right.rename(columns = {'券商.1':'券商','價格.1':'價格','買進股數.1':'買進股數','賣出股數\r':'賣出股數'})

In [ ]:
combine_df = pd.concat([df_left, df_right], axis=0)
combine_df = combine_df.sort_values(["券商"], ascending=True, ignore_index=True)
combine_df = combine_df.dropna(axis=0, how='any')
print(combine_df.head())

     券商     價格  買進股數   賣出股數
0  ?亞網路  103.5     0   3000
1  ?亞網路  103.0  1000   1000
2  ?亞鑫豐  102.5  6000   1000
3  ?亞鑫豐  102.0  3000      0
4  ?亞鑫豐  103.5     0  30000


In [ ]:
combine_df['價格'] = combine_df['價格'].astype(float)
combine_df['買進股數'] = combine_df['買進股數'].astype(float)
combine_df['賣出股數'] = combine_df['賣出股數'].astype(float)
print(combine_df)

        券商     價格    買進股數     賣出股數
0     ?亞網路  103.5     0.0   3000.0
1     ?亞網路  103.0  1000.0   1000.0
2     ?亞鑫豐  102.5  6000.0   1000.0
3     ?亞鑫豐  102.0  3000.0      0.0
4     ?亞鑫豐  103.5     0.0  30000.0
...    ...    ...     ...      ...
2043  高橋中壢  102.5  1000.0      0.0
2044  高橋內壢  103.0     0.0   5000.0
2045  高橋龍潭  103.0  1000.0      0.0
2046  高橋龍潭  102.5  1000.0      0.0
2047  高橋龍潭  103.5  2000.0   3000.0

[2048 rows x 4 columns]


In [ ]:
# 針對買進、賣出、買超、均買價、均賣價進行計算
temp = pd.DataFrame()
# 計算買進張數，每張為買進股數之千分之一
temp["買進"] = combine_df.groupby(by='券商').apply(lambda x: (x['買進股數']/1000).sum())
# 計算賣出張數，每張為賣出股數之千分之一
temp["賣出"] = combine_df.groupby(by='券商').apply(lambda x: (x['賣出股數']/1000).sum())
# 計算買超張數，買超為(買進股數-賣出股數)之千分之一
temp["買超"] = combine_df.groupby(by='券商').apply(lambda x: (x['買進股數']/1000).sum()- (x['賣出股數']/1000).sum())
#計算均買價，均買價為當日買進總價除以總股數
temp["均買價"] = combine_df.groupby(by='券商').apply(lambda x: (x['價格']*x['買進股數']/1000).sum()/((x['買進股數']/1000).sum()))
# 計算均賣價，均賣價為當日賣出總價除以總股數
temp["均賣價"] = combine_df.groupby(by='券商').apply(lambda x: (x['價格']*x['賣出股數']/1000).sum()/((x['賣出股數']/1000).sum()))
print(temp.head())

          買進       賣出       買超         均買價         均賣價
券商                                                    
?亞網路   1.000    4.000   -3.000  103.000000  103.375000
?亞鑫豐  34.300   49.000  -14.700  102.825073  103.295918
上海匯豐   2.359  340.000 -337.641  103.000000  102.863235
中信三重   0.002    1.104   -1.102  103.000000  103.452899
中信中壢  19.000   20.000   -1.000  102.105263  103.150000


In [ ]:
temp = temp.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
# 利用filter取出買超為正的row
positive = temp[temp['買超'] >= 0]
positive = positive.sort_values(by='買超', ascending=False).reset_index()

In [ ]:
negative = temp[temp['買超'] <= 0]
negative = negative.sort_values(by='買超').reset_index()

# negative = negative.sort_values(["券商"], ascending=True, ignore_index=True)

In [ ]:
del positive['index']
del negative['index']




In [ ]:
result = pd.concat([positive, negative], axis=1).round(0)
# result.set_axis(['券商', '買進', '賣出', '買超', '均買價', '均賣價']*2, axis=1, inplace=True)
print(result)

       券商      買進     賣出      買超    均買價    均賣價    券商      買進      賣出      買超  \
0    台灣匯立  3517.0    0.0  3517.0  103.0    NaN  美商高盛   302.0  2972.0 -2670.0   
1    瑞士信貸  3657.0  629.0  3028.0  103.0  103.0    富邦     1.0  1575.0 -1574.0   
2    台灣摩根  1345.0  460.0   885.0  103.0  103.0    瑞銀  2414.0  3005.0  -591.0   
3      元大  1357.0  534.0   823.0  103.0  103.0  合庫台中    16.0   514.0  -497.0   
4    凱基台北   961.0  505.0   456.0  103.0  103.0  花旗環球   328.0   804.0  -476.0   
..    ...     ...    ...     ...    ...    ...   ...     ...     ...     ...   
482   NaN     NaN    NaN     NaN    NaN    NaN  元大北港     4.0     4.0     0.0   
483   NaN     NaN    NaN     NaN    NaN    NaN  聯邦忠孝     1.0     1.0     0.0   
484   NaN     NaN    NaN     NaN    NaN    NaN  致和金華     2.0     2.0     0.0   
485   NaN     NaN    NaN     NaN    NaN    NaN  元富台南     2.0     2.0     0.0   
486   NaN     NaN    NaN     NaN    NaN    NaN  土銀嘉義     3.0     3.0     0.0   

       均買價    均賣價  
0    103.0  103.0  

In [ ]:
result.to_csv("1021_湯傑堯_M11015080_result.csv", encoding='big5')